In [162]:
!pip install nltk gensim

^C
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
import time
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from gensim.parsing.preprocessing import STOPWORDS, strip_tags, strip_numeric, strip_punctuation, strip_multiple_whitespaces, remove_stopwords
from groq import Groq
import os
from dotenv import load_dotenv
from nltk.tokenize import sent_tokenize
import json
from openai import OpenAI

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/oskarroeske/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# EDA

In [ ]:
df_reports = pd.read_csv("preprocessed_reports.csv")
df_paragraphs = pd.read_csv("preprocessed_paragraphs.csv")

In [ ]:
df_paragraphs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10571 entries, 0 to 10570
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    10571 non-null  int64 
 1   document_id   10571 non-null  int64 
 2   paragraph_id  10571 non-null  int64 
 3   paragraph     10571 non-null  object
dtypes: int64(3), object(1)
memory usage: 330.5+ KB


In [ ]:
def clean_text_again(text):
    text = strip_tags(text)
    text = strip_multiple_whitespaces(text)  # Normalize whitespaces
    return text

In [ ]:
df_paragraphs["paragraph"] = df_paragraphs["paragraph"].apply(clean_text_again)

# LLM Argument Mining

- Provide List of pre-defined arguments

In [ ]:
list_of_arguments = [
    "Earnings per Share",
    "Cash Flow",
    "Revenue",
    "Return On Equity",
    "Margins",
    "Cost Management",
    "Dividend Policy",
    "Investments",
    "Balance Sheet",
    "Long-term Growth",
    "Merges and Acquisition",
    "Refranchising",
    "Sustainability",
    "Employees",
    "Research and Development",
    "Marketing",
    "Shares Repurchase",
    "Processes",
    "Leadership",
    "Innovation",
    "Product Characteristics",
    "Pricing Strategy",
    "Production",
    "Technology Trends",
    "Market Share",
    "Market Expansion",
    "Competitors",
    "Global Presence",
    "Industry Outlook",
    "Regulations",
    "Partnerships and Collaborations",
    "Supply Chain",
    "Economic Conditions",
    "Demand",
    "Customers"
]


In [ ]:
def find_arguments(text, api_key):
    client = Groq(api_key=api_key)

    categories_string = ', '.join(f'"{cat}"' for cat in list_of_arguments)

    system_content = f"""You will classify financial sentences into predefined categories or create a new one if needed:
- Read the whole instructions first and then the provided text.
- Assign arguments to the best-matching category. 
- Try to use the existing categories first, only create a new category if no existing fits.
- New categories must be generic, concise and short ()maximum 2 words).
- IGNORE brand names/target prices/recommendations/valuation as arguments.
- Include new categories in the output
- Classify each argument as 'positive', 'negative', or 'neutral' from a financial perspective.
- Return the arguments and sentiment as a dictionary (e.g., {{'Category 1': 'positive', 'Category 2': 'negative'}}'
- Return only the JSON dictionary, no extra text.
    """
    final_prompt = f"""
        Existing Categories:[{categories_string}]
        Text: {text}
       """
    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_content,
            },
            {
                "role": "user",
                "content": final_prompt,
            }
        ],
        temperature=0,
        model="llama-3.3-70b-versatile",
    )
    #print(response)

    return response.choices[0].message.content

In [ ]:
import json
from itertools import cycle

# Load API key
load_dotenv()

api_key = os.getenv("GROQ_API_KEY")

def extract_arguments_and_sentiment(commentary):
    global list_of_arguments  # Ensure we can modify the global categories list
    
    # Call the find_arguments function with the selected API key
    result_json = find_arguments(commentary,api_key=api_key)   
    
    try:
        # Parse the JSON response
        result_dict = json.loads(result_json)
        
        # Check for new categories and add them to the global list
        for category in result_dict.keys():
            if category not in list_of_arguments:
                list_of_arguments.append(category)  # Dynamically update the categories list
        
        time.sleep(11)
        return result_dict  # Return the result
    except json.JSONDecodeError:
        print("Failed to decode JSON response.")
        return None


In [ ]:
df_paragraphs

,Unnamed: 0,document_id,paragraph_id,paragraph
0,0,1,1,: $150.00 We raise our estimates for 3Q16 as w...
1,1,1,2,Our $150 target price embeds a 10-year OIBDA g...
2,2,1,3,"FB is the largest social network, with 1.65B u..."
3,3,1,4,Risks to our thesis and target price include a...
4,4,2,1,FB reported relatively strong 2Q resultsrevenu...
...,...,...,...,...
10566,10566,923,2,Investors will need to consider the following ...
10567,10567,923,3,Much of online advertising is highly competiti...
10568,10568,923,4,SMEs have been the core . We believe they have...
10569,10569,923,5,A looming threat for all web publishers relate...


In [ ]:
df_paragraphs["provided_arguments"] = df_paragraphs["paragraph"].apply(extract_arguments_and_sentiment)

ChatCompletion(id='chatcmpl-acbeda7e-badc-4113-bfb4-785b4a5f5f54', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"Revenue": "positive", "Earnings per Share": "positive", "Market Expansion": "positive", "Investments": "positive", "Advertising": "positive", "Growth": "positive", "Cash Flow": "positive", "Margins": "positive"}', role='assistant', function_call=None, tool_calls=None))], created=1734199428, model='llama-3.3-70b-versatile', object='chat.completion', system_fingerprint='fp_fcc3b74982', usage=CompletionUsage(completion_tokens=56, prompt_tokens=887, total_tokens=943, completion_time=0.203636364, prompt_time=0.095212829, queue_time=0.0022593800000000053, total_time=0.298849193), x_groq={'id': 'req_01jf35egwzes3txn33ym9sbaf2'})
ChatCompletion(id='chatcmpl-5a5206b5-e451-431b-9675-ca4c835b1f63', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"Growth": "positive"}', role='a

KeyboardInterrupt: 

In [ ]:
df_paragraphs

,Unnamed: 0,document_id,paragraph_id,paragraph,provided_arguments
0,0,1,1,: $150.00 We raise our estimates for 3Q16 as w...,"{'Revenue': 'positive', 'Earnings per Share': ..."
1,1,1,2,Our $150 target price embeds a 10-year OIBDA g...,{'Long-term Growth': 'positive'}
2,2,1,3,"FB is the largest social network, with 1.65B u...","{'Market Share': 'positive', 'Customer': 'posi..."
3,3,1,4,Risks to our thesis and target price include a...,"{'Regulations': 'negative', 'Market Expansion'..."
4,4,2,1,FB reported relatively strong 2Q resultsrevenu...,"{'Revenue': 'positive', 'Earnings per Share': ..."
5,5,2,2,2020 revenue/GAAP EPS move to $79.0B/$7.51 fro...,"{'Revenue': 'positive', 'Earnings per Share': ..."
6,6,2,3,180MM mainly SMBs use FBs tools to find and co...,"{'Market Expansion': 'positive', 'Advertising'..."


In [ ]:
import pandas as pd
import ast

# Initialize a list to store the exploded rows
rows = []

# Iterate through the DataFrame and expand the dictionary into rows
for index, row in df_paragraphs.iterrows():
    provided_args = row["provided arguments"]
    # Convert string to dictionary if needed
    if isinstance(provided_args, str):
        try:
            provided_args = ast.literal_eval(provided_args)  # Convert string to dictionary
        except (ValueError, SyntaxError):
            print(f"Invalid format for provided_args at index {index}: {provided_args}")
            continue  # Skip invalid entries

    # Ensure it's a dictionary
    if isinstance(provided_args, dict):
        #print(f"Processed provided_args: {provided_args}")
        for argument, sentiment in provided_args.items():
            rows.append({
                "ID": row["ID"],
                "Argument": argument,
                "Sentiment": sentiment
            })

# Create a new DataFrame from the exploded rows
df_result = pd.DataFrame(rows)

# Display the resulting DataFrame
print(df_result)

     ID             Argument Sentiment
0     1          Solid 1QF20  positive
1     1     Long-term Growth  positive
2     1  Channel Development  positive
3     1     Positive Traffic  positive
4     1   Buying Opportunity  positive
..   ..                  ...       ...
845  20     Market Expansion  positive
846  20        US EV credits  positive
847  20   raw material costs  negative
848  20        US regulation  negative
849  20      Model 3Y demand  negative

[850 rows x 3 columns]


In [ ]:
df_result.info()

NameError: name 'df' is not defined

In [ ]:
import pandas as pd
import json
from collections import Counter, defaultdict

# Initialize a dictionary to hold counts for each key and sentiment
sentiment_counts = defaultdict(lambda: Counter({'positive': 0, 'neutral': 0, 'negative': 0}))

# Loop through each entry in the provided_arguments column
for entry in df_paragraphs_testing["provided arguments"]:
    # Ensure the entry is a dictionary
    if isinstance(entry, str):
        try:
            # Convert string to dictionary if it's in JSON-like format
            arguments_dict = json.loads(entry.replace("'", '"'))  # Replace single quotes with double quotes for JSON
        except json.JSONDecodeError:
            continue  # Skip any rows that can't be parsed as dictionaries
    elif isinstance(entry, dict):
        arguments_dict = entry
    else:
        continue  # Skip if entry is not a dictionary or parseable string

    # Update counts for each key and sentiment
    for key, sentiment in arguments_dict.items():
        sentiment_counts[key][sentiment] += 1

# Convert the result to a DataFrame for easier viewing
sentiment_overview_df = pd.DataFrame(sentiment_counts).T
sentiment_overview_df.columns = ["positive", "neutral", "negative"]
sentiment_overview_df.index.name = "Argument"

# Display the overview DataFrame
sentiment_overview_df.reset_index(inplace=True)
sentiment_overview_df.sort_values(by=["positive","negative","neutral"],ascending=False)


,Argument,positive,neutral,negative
5,Financial Guidance,6,2,0
0,Financial Performance,5,2,0
6,Share Repurchase,5,1,0
24,Revenue Growth,5,1,0
2,Capital Management,4,0,0
1,Dividend Policy,3,1,1
3,Earnings Guidance,3,1,0
10,Operational Efficiency,3,1,0
31,Product Strategy,3,1,0
32,Growth Strategy,3,0,0


# Check amount of intensifiers

In [ ]:
import re

# Function to count intensifiers in a single text
def count_intensifiers(text, intensifiers):
    # Convert to lowercase to make it case-insensitive
    words = re.findall(r'\b\w+\b', text.lower())
    return sum(words.count(word) for word in intensifiers)

In [ ]:
# Apply the function to the "Sentence" column of the DataFrame
df_paragraphs_testing["Basic Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=basic_intensifiers)
df_paragraphs_testing["Strong Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=strong_intensifiers)
df_paragraphs_testing["Formal Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=formal_emotive_intensifiers)

/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_75223/694084076.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_testing["Basic Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifiers=basic_intensifiers)
/var/folders/2h/tyzkflxs465698k6z8l3ktj00000gn/T/ipykernel_75223/694084076.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_paragraphs_testing["Strong Intensifier"] = df_paragraphs_testing["paragraphs"].apply(count_intensifiers, intensifie

In [ ]:
df_paragraphs_testing

,Unnamed: 0,ID,paragraphs,provided arguments,Basic Intensifier,Strong Intensifier,Formal Intensifier
0,0,1,We maintain our Overweight rating and raise ou...,"{'Financial Performance': 'positive', 'Dividen...",0,0,0
1,1,1,We remain in the minority with less than a thi...,"{'Balance Sheet': 'negative', 'Dividend Policy...",1,0,0
2,2,1,"Adjustments reect Q3 2021 actuals, our updated...","{'Operational Efficiency': 'positive', 'Capita...",0,0,0
3,3,1,XOM expects upstream volumes to increase on lo...,"{'Operational Efficiency': 'neutral', 'Upstrea...",0,0,0
4,4,1,XOM announced a 10bn share repurchase Buybacks...,"{'Share Repurchase': 'positive', 'Dividend Pol...",0,0,0
5,5,1,XOM announced annual capex range of 2025bn goi...,"{'Capital Management': 'positive', 'Cost Struc...",1,1,0
6,6,1,EPS of 1.57 was in betweenour estimate of 1.58...,"{'Financial Performance': 'neutral', 'Operatio...",0,0,0
7,7,1,Our 75 price target is based on 6.5x our 2023 ...,"{'Valuation': 'neutral', 'Earnings Guidance': ...",0,0,0
8,8,1,Systematic risks to our targetrating include c...,"{'Regulatory Environment': 'negative', 'Operat...",0,0,0
9,9,1,XOM We have an Overweight rating on XOM. Opex ...,"{'Financial Performance': 'positive', 'Cost Ma...",0,0,0


# paragraph removed from instructions: - New categories must be generic, concise (max 3 words), and included in the output for list updates.
""""You will classify financial text into predefined categories or create a new one if needed:\n",
    "- Assign arguments to the best-matching category. Create a new category only if no match fits.\n",
    "- IGNORE brand names/Target Prices/Recommendations/Valuation as arguments.\n",
    "- Classify each argument as positive, negative, or neutral.\n",
    "- Include new categories in the output.\n",
    "-Return the arguments and sentiment as a dictionary (e.g., {{'Category 1': 'positive', 'Category 2': 'negative'}}'\n",
    "- Return only the JSON dictionary, no extra text.\n",
    "Categories: [{categories_string}]\n",

In system_content: Categories: [{categories_string}]
"""

# Create client
load_dotenv()
api_key = os.getenv("LLAMA_API_KEY")

client = OpenAI(
    api_key = api_key,
    base_url = "https://api.llama-api.com"
    )


def find_arguments(text):
    categories_string = ', '.join(f'"{cat}"' for cat in list_of_arguments)

    system_content = f"""
Classify financial sentences into these categories: [{categories_string}]. Follow these rules:
1. Assign arguments to existing categories; create a new one only if no match exists (max 2 words, generic).
2. Ignore Target Price, brand names, recommendations, valuation.
3. If no argument exists, return {{}}.
4. Label each argument as 'positive', 'negative', or 'neutral'.
5. Output only a JSON dictionary, e.g., {{'Category': 'positive'}}.
""" 
    final_prompt = f"""
        Text: {text}
       """

    response = client.chat.completions.create(
        messages=[
            {
                "role": "system",
                "content": system_content,
            },
            {
                "role": "user",
                "content": final_prompt,
            }
        ],
        temperature=0,
        model="llama3-70b",
    )
    print(response)

    return response.choices[0].message.content